In [46]:
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import torch
from omegaconf import OmegaConf
from train_utils import prepare_data, prepare_model, prepare_optimizer, train

In [48]:
args = OmegaConf.create(dict(
        # General
        seed=0,
        # Data folders
        image_folder='datasets/train/images/',
        gt_folder='datasets/train/groundtruth/',
        # Data tranforms
        # random_resized_crop: crop a random portion of image and resize it to a given size
        random_resized_crop=False,
        output_size=(400, 400),  # expected output size of the crop, for each edge.
        random_resized_crop_scale=(0.5, 1.0),
        # random_horizontal_flip: randomly flip the image horizontally with a given probability
        random_horizontal_flip=False,
        # random_vertical_flip: randomly flip the image vertically with a given probability
        random_vertical_flip=False,
        # random_rotation: randomly rotate the image with a given probability
        random_rotation=False,
        degrees=10,  # range of degrees to select from
        # color_jitter: randomly change the brightness, contrast and saturation of an image
        color_jitter=False,
        brightness=0.1,  # how much to jitter brightness.
        contrast=0.1,  # how much to jitter contrast.
        saturation=0.1,  # how much to jitter saturation.
        hue=0.1,  # how much to jitter hue.
        # normalization
        normalization=True,  # TODO: Should it always be True?
        # Data loaders
        batch_size=4,
        train_size=0.8,
        val_size=0.1,
        test_size=0.1,
        # Model
        # UNetV1
        model_name='UNetV1',
        model_pretrained=False,
        model_scale=0.5,
        model_save_name='checkpoints/unetv1-1',
        # Optimizer
        optim_name='sgd',
        optim_lr=0.1,
        optim_momentum=0.9, # TODO: Try with (optim_momentum != 0) and without (momentum = 0)?
        # Training
        n_steps=2000,
        eval_freq = 100,
        # Wandb logging
        wandb_project="road-segmentation",
        wandb_run="run1"
        ))

In [49]:
# prepare train, validation and test loaders
train_loader, val_loader, test_loader = prepare_data(args)

Using ToTensor.
Using Normalize with mean=(0.5, 0.5, 0.5) and std=(0.5, 0.5, 0.5).


In [51]:
# prepare model
model = prepare_model(args)

Initializing UNetV1 model with pretrained=False, scale=0.5.


Using cache found in /home/nadezhda/.cache/torch/hub/milesial_Pytorch-UNet_master


In [52]:
# define loss function
criterion = torch.nn.BCEWithLogitsLoss()

In [53]:
# prepare optimizer
optimizer = prepare_optimizer(model, args) 

Initializing SGD optimizer with lr=0.1, momentum=0.9.


In [54]:
# device to use for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

UNetV1(
  (model): UNet(
    (inc): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (down1): Down(
      (maxpool_conv): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): DoubleConv(
          (double_conv): Sequential(
            (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
            (3): Conv2d(128, 128, kernel_

In [ ]:
# train
trained_model = train(model, device, train_loader, val_loader, criterion, optimizer, args)